## **Proyecto de Predicción de Churn en Telecomunicaciones - Grupo 3**

# **1. Configuración del Ambiente**

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

In [ ]:
global datos_churn

# **2. Obtención y Tratamiento de Datos**

## **2.1 Cargando las bases de datos**

In [ ]:
datos_churn = pd.read_json("base_clientes.json")
datos_churn.head()

,id_cliente,Churn,cliente,telefono,internet,cuenta
0,0002-ORFBO,no,"{'genero': 'femenino', 'anciano': 0, 'pareja':...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'DSL', 'seguridad_online...","{'contrato': None, 'facturacion_electronica': ..."
1,0003-MKNFE,no,"{'genero': 'masculino', 'anciano': 0, 'pareja'...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'DSL', 'seguridad_online...","{'contrato': 'mensual', 'facturacion_electroni..."
2,0004-TLHLJ,si,"{'genero': 'masculino', 'anciano': 0, 'pareja'...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'fibra optica', 'segurid...","{'contrato': 'mensual', 'facturacion_electroni..."
3,0011-IGKFF,si,"{'genero': 'masculino', 'anciano': 1, 'pareja'...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'fibra optica', 'segurid...","{'contrato': 'mensual', 'facturacion_electroni..."
4,0013-EXCHZ,si,"{'genero': 'femenino', 'anciano': 1, 'pareja':...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'fibra optica', 'segurid...","{'contrato': 'mensual', 'facturacion_electroni..."


In [ ]:
datos_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7344 entries, 0 to 7343
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  7344 non-null   object
 1   Churn       7344 non-null   object
 2   cliente     7344 non-null   object
 3   telefono    7344 non-null   object
 4   internet    7344 non-null   object
 5   cuenta      7344 non-null   object
dtypes: object(6)
memory usage: 344.4+ KB


In [ ]:
def lectura_datos():
  global datos_churn

  with open("base_clientes.json", encoding='utf-8') as f:
    json_bruto = json.load(f)
  datos_churn = pd.json_normalize(json_bruto)

In [ ]:
lectura_datos()

In [ ]:
datos_churn.head(20)

,id_cliente,Churn,cliente.genero,cliente.anciano,cliente.pareja,cliente.dependientes,cliente.tiempo_servicio,telefono.servicio_telefono,telefono.varias_lineas,internet.servicio_internet,internet.seguridad_online,internet.backup_online,internet.proteccion_dispositivo,internet.soporte_tecnico,internet.tv_streaming,internet.peliculas_streaming,cuenta.contrato,cuenta.facturacion_electronica,cuenta.metodo_pago,cuenta.cobros.mensual,cuenta.cobros.Total
0,0002-ORFBO,no,femenino,0,si,si,9.0,si,no,DSL,no,si,no,si,si,no,None,None,None,NaN,None
1,0003-MKNFE,no,masculino,0,no,no,9.0,si,si,DSL,no,no,no,no,no,si,mensual,no,cheque,59.90,542.4
2,0004-TLHLJ,si,masculino,0,no,no,4.0,si,no,fibra optica,no,no,si,no,no,no,mensual,si,cheque electronico,73.90,280.85
3,0011-IGKFF,si,masculino,1,si,no,13.0,si,no,fibra optica,no,si,si,no,si,si,mensual,si,cheque electronico,98.00,1237.85
4,0013-EXCHZ,si,femenino,1,si,no,3.0,si,no,fibra optica,no,no,no,si,si,no,mensual,si,cheque,83.90,267.4
5,0013-MHZWF,no,femenino,0,no,si,9.0,si,no,DSL,no,no,no,si,si,si,mensual,si,tarjeta de credito (automatico),69.40,571.45
6,0013-SMEOE,no,femenino,1,si,no,71.0,si,no,fibra optica,si,si,si,si,si,si,dos años,si,transferencia bancaria (automatica),109.70,7904.25
7,0014-BMAQU,no,masculino,0,si,no,63.0,si,si,fibra optica,si,no,no,si,no,no,dos años,si,tarjeta de credito (automatico),84.65,5377.8
8,0015-UOCOJ,no,femenino,1,no,no,7.0,si,no,DSL,si,no,no,no,no,no,mensual,si,cheque electronico,48.20,340.35
9,0016-QLJIS,no,femenino,0,si,si,NaN,si,si,DSL,si,si,si,si,si,si,dos años,si,cheque,90.45,5957.9


## **2.2 Tratamiento de Datos**



### Revisión de Diccionario

In [ ]:
with open('/content/Diccionario.txt', 'r', encoding='utf-8') as file:
    contenido = file.read()
print(contenido)

La base de datos contiene columnas además del ID de los clientes y el churn:

Cliente:
género: género (masculino y femenino)
anciano: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
pareja: si el cliente tiene o no una pareja
dependientes: si el cliente tiene o no dependientes
tiempo_servicio: meses de contrato del cliente

Servicio de telefonía:
servicio_telefono: suscripción al servicio telefónico
varias_lineas: suscripción a más de una línea telefónica

Servicio de internet:
servicio_internet: suscripción a un proveedor de internet
seguridad_online: suscripción adicional a seguridad en línea
backup_online: suscripción adicional a copias de seguridad en línea
proteccion_dispositivo: suscripción adicional a protección en el dispositivo
soporte_tecnico: suscripción adicional a soporte técnico, menos tiempo de espera
tv_streaming: suscripción a TV por cable
peliculas_streaming: suscripción a streaming de películas

Cuenta:
contrato: tipo de contrato
factura_e

### Análisis Exploratorio de los Datos

In [ ]:
datos_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7344 entries, 0 to 7343
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id_cliente                       7344 non-null   object 
 1   Churn                            7344 non-null   object 
 2   cliente.genero                   7344 non-null   object 
 3   cliente.anciano                  7344 non-null   int64  
 4   cliente.pareja                   7344 non-null   object 
 5   cliente.dependientes             7344 non-null   object 
 6   cliente.tiempo_servicio          7336 non-null   float64
 7   telefono.servicio_telefono       7344 non-null   object 
 8   telefono.varias_lineas           7344 non-null   object 
 9   internet.servicio_internet       7344 non-null   object 
 10  internet.seguridad_online        7344 non-null   object 
 11  internet.backup_online           7344 non-null   object 
 12  internet.proteccion_

In [ ]:
datos_churn.shape

(7344, 21)

In [ ]:
datos_churn.describe()

,cliente.anciano,cliente.tiempo_servicio,cuenta.cobros.mensual
count,7344.000000,7336.000000,7326.000000
mean,0.162990,33.271265,64.683770
std,0.369382,35.776684,30.143033
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.362500
50%,0.000000,29.000000,70.300000
75%,0.000000,56.000000,89.887500
max,1.000000,1080.000000,118.750000


In [ ]:
#Revisión de valores nulos en las columnas
datos_churn.isnull().sum()

,0
id_cliente,0
Churn,0
cliente.genero,0
cliente.anciano,0
cliente.pareja,0
cliente.dependientes,0
cliente.tiempo_servicio,8
telefono.servicio_telefono,0
telefono.varias_lineas,0
internet.servicio_internet,0


In [ ]:
datos_churn.columns

Index(['id_cliente', 'Churn', 'cliente.genero', 'cliente.anciano',
       'cliente.pareja', 'cliente.dependientes', 'cliente.tiempo_servicio',
       'telefono.servicio_telefono', 'telefono.varias_lineas',
       'internet.servicio_internet', 'internet.seguridad_online',
       'internet.backup_online', 'internet.proteccion_dispositivo',
       'internet.soporte_tecnico', 'internet.tv_streaming',
       'internet.peliculas_streaming', 'cuenta.contrato',
       'cuenta.facturacion_electronica', 'cuenta.metodo_pago',
       'cuenta.cobros.mensual', 'cuenta.cobros.Total'],
      dtype='object')

In [ ]:
#Revisión de registros vacios en cada columna
registros_vacios = datos_churn.apply(lambda col: col.astype(str).str.strip().eq('').sum())
registros_vacios

,0
id_cliente,0
Churn,226
cliente.genero,0
cliente.anciano,0
cliente.pareja,0
cliente.dependientes,0
cliente.tiempo_servicio,0
telefono.servicio_telefono,0
telefono.varias_lineas,0
internet.servicio_internet,0


In [ ]:
#revisando los registros en de la columna Churn
datos_churn[datos_churn['Churn'] == '']

,id_cliente,Churn,cliente.genero,cliente.anciano,cliente.pareja,cliente.dependientes,cliente.tiempo_servicio,telefono.servicio_telefono,telefono.varias_lineas,internet.servicio_internet,internet.seguridad_online,internet.backup_online,internet.proteccion_dispositivo,internet.soporte_tecnico,internet.tv_streaming,internet.peliculas_streaming,cuenta.contrato,cuenta.facturacion_electronica,cuenta.metodo_pago,cuenta.cobros.mensual,cuenta.cobros.Total
30,0047-ZHDTW,,femenino,0,no,no,11.0,si,si,fibra optica,si,no,no,no,no,no,mensual,si,transferencia bancaria (automatica),79.00,929.3
75,0120-YZLQA,,masculino,0,no,no,71.0,si,no,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,si,tarjeta de credito (automatico),19.90,1355.1
96,0154-QYHJU,,masculino,0,no,no,29.0,si,no,DSL,si,si,no,si,no,no,un año,si,cheque electronico,58.75,1696.2
98,0162-RZGMZ,,femenino,1,no,no,5.0,si,no,DSL,si,si,no,si,no,no,mensual,no,tarjeta de credito (automatico),59.90,287.85
175,0274-VVQOQ,,masculino,1,si,no,65.0,si,si,fibra optica,no,si,si,no,si,si,un año,si,transferencia bancaria (automatica),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7211,9920-GNDMB,,masculino,0,no,no,9.0,si,si,fibra optica,no,no,no,no,no,no,mensual,si,cheque electronico,76.25,684.85
7239,9955-RVWSC,,femenino,0,si,si,67.0,si,no,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,si,transferencia bancaria (automatica),19.25,1372.9
7247,9966-VYRTZ,,femenino,0,si,si,31.0,si,no,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,mensual,si,cheque,19.55,658.95
7267,6532-YOHZY,,masculino,0,si,si,45.0,si,si,fibra optica,no,si,si,si,si,si,dos años,si,transferencia bancaria (automatica),109.75,4900.65


In [ ]:
#revisando los registros en de la columna cuenta.cobros.Total
datos_churn[datos_churn['cuenta.cobros.Total'] ==  ' ']

,id_cliente,Churn,cliente.genero,cliente.anciano,cliente.pareja,cliente.dependientes,cliente.tiempo_servicio,telefono.servicio_telefono,telefono.varias_lineas,internet.servicio_internet,internet.seguridad_online,internet.backup_online,internet.proteccion_dispositivo,internet.soporte_tecnico,internet.tv_streaming,internet.peliculas_streaming,cuenta.contrato,cuenta.facturacion_electronica,cuenta.metodo_pago,cuenta.cobros.mensual,cuenta.cobros.Total
975,1371-DWPAZ,no,femenino,0,si,si,0.0,no,sin servicio de telefono,DSL,si,si,si,si,si,no,dos años,no,tarjeta de credito (automatico),56.05,
1775,2520-SGTTA,no,femenino,0,si,si,0.0,si,no,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,no,cheque,20.00,
1955,2775-SEFEE,no,masculino,0,no,si,0.0,si,si,DSL,si,si,no,si,no,no,dos años,si,transferencia bancaria (automatica),61.90,
2075,2923-ARZLG,no,masculino,0,si,si,0.0,si,no,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,si,cheque,19.70,
2232,3115-CZMZD,no,masculino,0,no,si,0.0,si,no,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,no,cheque,20.25,
2308,3213-VVOLG,no,masculino,0,si,si,0.0,si,si,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,no,cheque,25.35,
2930,4075-WKNIU,no,femenino,0,si,si,0.0,si,si,DSL,no,si,si,si,si,no,dos años,no,cheque,73.35,
3134,4367-NUYAO,no,masculino,0,si,si,0.0,si,si,no,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,sin servicio de internet,dos años,no,cheque,25.75,
3203,4472-LVYGI,no,femenino,0,si,si,0.0,no,sin servicio de telefono,DSL,si,no,si,si,si,no,dos años,si,transferencia bancaria (automatica),52.55,
4169,5709-LVOEQ,no,femenino,0,si,si,0.0,si,no,DSL,si,si,si,no,si,si,dos años,no,cheque,80.85,


### Generación de la función Preprocesamiento

In [ ]:
def preprocesamiento():
  global datos_churn

  print(f'Cantidad de registros en el dataset: {datos_churn.shape[0]}')

  #identificación de índices con valores vacios en la columna 'cuenta.cobros.Total'
  idx = datos_churn[datos_churn['cuenta.cobros.Total'] ==  ' '].index
  #se llena los registros vacios identificados multiplicando los valores de 'cuenta.cobros.mensual'*24
  datos_churn.loc[idx, 'cuenta.cobros.Total'] = datos_churn.loc[idx, 'cuenta.cobros.mensual']*24
  #asignación a tiempo de servicio igual a 24 para aquellos indices identificados
  datos_churn.loc[idx, 'cliente.tiempo_servicio'] = datos_churn.loc[idx, 'cliente.tiempo_servicio'].replace(0, 24)

  #se convierte a columna numérica y convierte a nulo todo valor que no sea numérico
  datos_churn['cuenta.cobros.Total'] = pd.to_numeric(datos_churn['cuenta.cobros.Total'], errors='coerce')
  #se cambia el tipo de datos a float
  datos_churn['cuenta.cobros.Total'] = datos_churn['cuenta.cobros.Total'].astype('float64')

  #seleccion de columnas de tipo objeto
  columna_objects = datos_churn.select_dtypes(include='object').columns
  #se reemplaza los registros vacios en las columnas seleccionadas por nulos
  datos_churn[columna_objects] = datos_churn[columna_objects].replace('', np.nan)
  #se eliminan los valores nulos en la columnas object
  datos_churn.dropna(subset=columna_objects, inplace=True)
  #se eliminan duplicados
  datos_churn.drop_duplicates(inplace=True)
  print(f'Cantidad de registros en el dataset luego de eliminar nulos y duplicados en las columnas: {datos_churn.shape[0]}')

  #relleno los valores nulos de la columna 'cliente.tiempo_servicio'
  datos_churn['cliente.tiempo_servicio'] = datos_churn['cliente.tiempo_servicio'].fillna(datos_churn['cuenta.cobros.Total'] / datos_churn['cuenta.cobros.mensual'])
  #se reinicia el indice del dataframe
  datos_churn = datos_churn.reset_index(drop=True)

  #Cálculo del rango intercuartílico (IQR)
  valor = datos_churn['cliente.tiempo_servicio']
  Q1 = float(valor.quantile(.25))
  Q3 = float(valor.quantile(.75))
  IQR = Q3 - Q1
  #determinación de los limites
  limite_inferior = Q1 - (1.5*IQR)
  limite_superior = Q3 + (1.5*IQR)
  # Identificar índices con valores outliers donde 'cliente.tiempo_servicio' es menor que 'limite_inferior' o mayor que 'limite_superior'
  outlier_indices = datos_churn[(datos_churn['cliente.tiempo_servicio'] < limite_inferior) | (datos_churn['cliente.tiempo_servicio'] > limite_superior)].index

  #Se recalcula el valor de 'cliente.tiempo_servicio' en los indices con outliers identificados dividiendo cuenta.cobros.Total por cuenta.cobros.mensual
  datos_churn.loc[outlier_indices, 'cliente.tiempo_servicio'] = datos_churn['cuenta.cobros.Total'] / datos_churn['cuenta.cobros.mensual']
  #Se vuelve a calcular el IQR luego de la corrección
  valor = datos_churn['cliente.tiempo_servicio']
  Q1 = float(valor.quantile(.25))
  Q3 = float(valor.quantile(.75))
  IQR = Q3 - Q1
  #se determina nuevos limites luego del recalculo
  limite_inferior = Q1 - (1.5*IQR)
  limite_superior = Q3 + (1.5*IQR)
  #se elimina los outliers del dataframe
  datos_churn = datos_churn[(datos_churn['cliente.tiempo_servicio'] >= limite_inferior) & (datos_churn['cliente.tiempo_servicio'] <= limite_superior)]
  #se reinicia el indice del dataframe
  datos_churn = datos_churn.reset_index(drop=True)

  print(f'Cantidad de registros en el dataset luego de eliminar registros con outliers en tiempo de servicio: {datos_churn.shape[0]}')


In [ ]:
preprocesamiento()

Cantidad de registros en el dataset: 7344
Cantidad de registros en el dataset luego de eliminar nulos y duplicados en las columnas: 7006
Cantidad de registros en el dataset luego de eliminar registros con outliers en tiempo de servicio: 7002


In [ ]:
datos_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7002 entries, 0 to 7001
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id_cliente                       7002 non-null   object 
 1   Churn                            7002 non-null   object 
 2   cliente.genero                   7002 non-null   object 
 3   cliente.anciano                  7002 non-null   int64  
 4   cliente.pareja                   7002 non-null   object 
 5   cliente.dependientes             7002 non-null   object 
 6   cliente.tiempo_servicio          7002 non-null   float64
 7   telefono.servicio_telefono       7002 non-null   object 
 8   telefono.varias_lineas           7002 non-null   object 
 9   internet.servicio_internet       7002 non-null   object 
 10  internet.seguridad_online        7002 non-null   object 
 11  internet.backup_online           7002 non-null   object 
 12  internet.proteccion_